# **Exercise1: ‘Data set’**

## Part 1: Dataset Import script:

Load the protein sequence data set in the .fasta format. The numbers in the square brackets correspond to class labels.
Hint: Search for the python package ‘biopython’

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.6 MB/s eta 0:00:00


In [1]:
##### Imports #####
from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

/home/lilly/.local/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


### **Data Import Function:** In this function we import the .fasta file using SeqIO module of the Bio package.

In [4]:
### Function to import fasta file
def get_fasta_file():
  fasta_file_path = "./protein_sequence.fasta"
  sequences = SeqIO.parse(fasta_file_path, "fasta")
  return sequences

In [5]:
### Get FASTA file
sequences = get_fasta_file()

### Iterate over first 5 sequences and print information
i = 1
for seq_record in sequences:
    if i > 5:
      break
    print("ID:", seq_record.id)
    print("Description:", seq_record.description)
    print("Sequence:", seq_record.seq)
    print("Length:", len(seq_record))
    print("--------------------------------------")
    i += 1

ID: Q5AV81
Description: Q5AV81 [16491]
Sequence: MVRIFITGSSDGIGQAAAKVLSEQGHSVVLHARNADRAASAQEAVPGAEAVLVGDLSSIAETKALAEEANKLPPFDTVIHNAGIGYGATASQEITADKISAVFAVNTLAPYILTCLMHKPKARLLYMSSDSHYGGDESLRNITQSHSYGNTKLHDVMLANAFSRRWGNAIQVVSMHPGWVRTKMGGVMAPGALDRPARVLADWAIGKGDLARLKSGTFFTTSGPESAHPGADNVQKQEELLRICKEVSGVGVPGG
Length: 255
--------------------------------------
ID: B1Y654
Description: B1Y654 [16491]
Sequence: MTHALDTLALNQLFTEARTHNAWRDEPVAPELLRQLHELAKWGPTAMNCCPLRVRFIVGAEARARLLPLMSEGNRAKTAAAPVIAILGRDVDFHQHMPTLAPHMAGARERFADQPAQREAMSQLNGALEAGYFILAARALGLDCGPMGGFDAAAVDAEFWAGTAVRSMIVCNLGHGDPAGLRPRAPRLDFDTACAVL
Length: 197
--------------------------------------
ID: P83288
Description: P83288 [90729]
Sequence: MKIVLVFVCTLYLAQATYLSEQDVNEVSEFLEALDQANEAASEMVEAAETEEARDWECLPLHSSCDNDCVCCKNHHCHCPYSNVSKLEKWLPEWAKIPDALKRCSCQRNDKDGKINTCDKYKN
Length: 123
--------------------------------------
ID: P39901
Description: P39901 [3677]
Sequence: MIALSVCWQIVRYLCRIRQMXGLNAIEHSNYLRNYMGHLRQKLEQDPARPRHFITETGIGYRFML
Length: 

## Part 2: Smith-Waterman Algorithm for Alignments

### **Smith-Waterman Alignment Score:**
This function is used to get the best alignment between two sequences. First we call the pairwise2.align.localms function which performs the pairwise sequence alignment using a dynamic programming algorithm.

When calling this function, we specify the match score and gap penalties. The match score indicates the compatibility between an alignment of two characters in the sequences. Highly compatible characters should be given positive scores, and incompatible ones should be given negative scores or 0. The gap penalties should be negative.

Then we get the alignment for first two sequences of the dataset and for a nice printout, we use the format_alignment method.



In [6]:
### Calculating Smith-Waterman alignment
def smith_waterman_alignment(seq1, seq2, match_score=2, mismatch_penalty=-2, gap_open_penalty=-1, gap_extend_penalty=-0.5):

    # The pairwise2.align.localms() function for local sequence alignment
    alignments = pairwise2.align.localms(seq1, seq2, match_score, mismatch_penalty, gap_open_penalty, gap_extend_penalty)
    best_alignment = max(alignments, key = lambda x: x.score)

    return best_alignment       #Return the best alignment

In [7]:
### SeqIO.parse() function to read the FASTA file
sequences = get_fasta_file()

sequence1 = next(sequences, None)
sequence2 = next(sequences, None)

print("First sequence: ", sequence1.id, ": ", sequence1.seq)
print("Second sequence: ", sequence2.id, ": ", sequence2.seq)

### Get and print the alignment
alignment = smith_waterman_alignment(sequence1.seq,sequence2.seq)
print(format_alignment(*alignment))

First sequence:  Q5AV81 :  MVRIFITGSSDGIGQAAAKVLSEQGHSVVLHARNADRAASAQEAVPGAEAVLVGDLSSIAETKALAEEANKLPPFDTVIHNAGIGYGATASQEITADKISAVFAVNTLAPYILTCLMHKPKARLLYMSSDSHYGGDESLRNITQSHSYGNTKLHDVMLANAFSRRWGNAIQVVSMHPGWVRTKMGGVMAPGALDRPARVLADWAIGKGDLARLKSGTFFTTSGPESAHPGADNVQKQEELLRICKEVSGVGVPGG
Second sequence:  B1Y654 :  MTHALDTLALNQLFTEARTHNAWRDEPVAPELLRQLHELAKWGPTAMNCCPLRVRFIVGAEARARLLPLMSEGNRAKTAAAPVIAILGRDVDFHQHMPTLAPHMAGARERFADQPAQREAMSQLNGALEAGYFILAARALGLDCGPMGGFDAAAVDAEFWAGTAVRSMIVCNLGHGDPAGLRPRAPRLDFDTACAVL
45 VPGAEAV---LVGDL---SSIAE-----TKALAEEANKLPPFDTVIHNAGI-G
   | ||||    |   |   |   |     | | |  |   |    ||  | | |
58 V-GAEA-RARL---LPLMS---EGNRAKT-A-A--A---P----VI--A-ILG
  Score=13.5



# **Exercise 2: ‘Embedding’**



## Part 1: Construct a pair-wise similarity matrix using the Smith-Waterman alignment

### **Smith-Waterman alignment for Best Alignment Score:**
Here, we create another implementation of the Smith Water Alignment to get the best alignment score instead of just the alignment. This score will be further used to create a Similarity Matrix.

In [ ]:
### Calculating Smith-Waterman alignment score
def smith_waterman_alignment_score(seq1, seq2, match_score=2, mismatch_penalty=-1, gap_open_penalty=-2, gap_extend_penalty=-0.5):

    # The pairwise2.align.localms() function for local sequence alignment
    alignments = pairwise2.align.localms(seq1, seq2, match_score, mismatch_penalty, gap_open_penalty, gap_extend_penalty)

    if bool(alignments):
      best_alignment = max(alignments, key = lambda x: x.score)
      return best_alignment.score
    else:
      best_alignment = 0
      return 0

    #return best_alignment.score       #Return the best alignment

### **Calculating Similarity Matrix:**
In this function we calculate the similarity matrix for the sequences. The parameter num_sequences stands for the number of sequences we want to compare and get the similarity matrix for. We first initialize the matrix with zeros, and then inside a double loop, we get the best alignment score of two sequences. Then we store that value in the similarity matrix and since we assume symmetry, we fill the upper as well as lower triangular part of the matrix with the same value.

In [ ]:
### Pair-wise similarity matrix
def get_similarity_matrix(sequences, num_sequences):
  k=1
  similarity_matrix = np.zeros((num_sequences,num_sequences))
  for i in range(num_sequences):
      for j in range(i, num_sequences):
          seq1 = sequences[i].seq
          seq2 = sequences[j].seq
          alignment_score = smith_waterman_alignment_score(seq1, seq2, 1, -1, -1,-1)

          # Assume symmetry
          similarity_matrix[i][j] = similarity_matrix[j][i] = alignment_score
      #print("\n",k)
      k+=1

  return similarity_matrix

### **Printing the Similarity Matrix:**
For computational efficiency, we sort the sequence objects in ascending order of their sequence values and get the top 250 sequences(out of the 5000 given). This ensures that the sequences which are smaller are used, which reduces the computational time.

We then call the function to get the similarity matrix and print it.

In [ ]:
### Importing the FASTA file
fasta_iterator = get_fasta_file()
sequences = list(fasta_iterator)

### Sorting the sequences in Ascending order and Taking the top 250 sequences
sorted_seq = sorted(sequences, key=lambda sequence: sequence.seq)
print("Sorted Sequence: ", sorted_seq[0].seq)
top250_seq = sorted_seq[0:250]
print("Taking ", len(top250_seq), "Sequences")
num_sequences = len(top250_seq)

### Pair-wise similarity matrix
similarity_matrix = get_similarity_matrix(top250_seq, num_sequences)

## Print the similarity matrix
print(similarity_matrix)

Sorted Sequence:  AACKCDDEGPDIRTAPLTGTVDLGSCNAGWEKCASYYTIIADCCRKKK
Taking  250 Sequences
[[ 48.   4.   3. ...   3.   3.   3.]
 [  4.  80.   2. ...   3.   3.   3.]
 [  3.   2.  45. ...   3.   2.   3.]
 ...
 [  3.   3.   3. ... 284.   5.   4.]
 [  3.   3.   2. ...   5.  86.   6.]
 [  3.   3.   3. ...   4.   6. 186.]]


## Part 2: Implement an embedding into a 100-dimensional euclidean vector space

### **Embedding into a 100-d space:**

After getting the Similarity Matrix, we get the first k-eigenvectors and eigenvalues(in this case first 100). So, what do the first k-eigenvectors stand for? Like all clustering algorithms, we must choose the number of clusters for our algorithm to find.

Then, we do a Matrix Multiplication of the similarity matrix and the computed top k-eigenvectors. Due to this, the similarity matrix gets transformed into the 100d space defined by the calculated k-eigenvectors. Finally we return this result as an embedding_matrix.


In [ ]:
### Function to do embedding
def get_embedding(similarity_matrix, num_dimensions=100):

    ### Compute Eigenvalues and Eigenvectors of similarity matrix
    eigenvalues, eigenvectors = np.linalg.eigh(similarity_matrix)

    ### Sort Eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_eigenvalues = eigenvalues[sorted_indices][:num_dimensions]

    ### Extract Eigenvectors
    top_eigenvectors = eigenvectors[:, sorted_indices][:, :num_dimensions]

    ### Form Embedding Matrix using Dot Product
    embedding_matrix = np.dot(similarity_matrix, top_eigenvectors)

    return embedding_matrix

In [ ]:
### embedding_matrix = get_embedding(similarity_matrix)
embedding_matrix = get_embedding(similarity_matrix)
print(embedding_matrix)
print(embedding_matrix.shape)

[[-1.92150937e+01  3.18304207e+01 -4.51358187e+00 ... -9.64705845e-01
  -6.20047342e-01  1.80334938e+00]
 [-2.10952741e+01  3.22535660e+01 -4.74864587e+00 ... -7.35705452e-01
   2.23203270e+00 -1.46672328e+00]
 [-1.71902700e+01  2.92446028e+01 -3.02794428e+00 ...  1.11449501e+00
  -1.32588048e+00 -4.28698868e+00]
 ...
 [-2.68815265e+01  4.32029452e+01 -7.68936076e+00 ... -5.72242379e-03
   1.65809321e-01  3.01997411e-01]
 [-2.15123580e+01  2.96911178e+01 -6.84675821e+00 ...  2.01830481e+00
   5.87557856e-01 -5.58509778e-01]
 [-2.26318624e+01  3.77595997e+01 -7.83863457e+00 ...  2.43191266e-01
   6.09600441e-01  8.30145011e-03]]
(250, 100)
